## Dados

Colunas que iremos usar:
- valor do frete [varia, depende cliente]
    - freight_value de order_items_dataset
- tempo de entrega [varia, depende cliente]
    - order_estimated_delivery_date - order_purchase_timestamp ambos de orders_dataset
- quantidade de fotos publicadas daquele produto [não varia]
    - product_photos_qty de products_dataset
- quantidade de caracteres na descrição do produto [não varia]
    - product_description_lenght de products_dataset
- avaliação do cliente sobre o produto [notas distribuidas para todos os produtos, nao varia]
    - review_score de reviews_dataset

In [1]:
import pandas as pd

df_reviews = pd.read_csv("Data\olist_order_reviews_dataset.csv")
df_order_items = pd.read_csv("Data\olist_order_items_dataset.csv")
df_orders = pd.read_csv("Data\olist_orders_dataset.csv")
df_products = pd.read_csv("Data\olist_products_dataset.csv")
df_customers = pd.read_csv("Data\olist_customers_dataset.csv")

In [2]:
# juntando em 1 dataframe
df = pd.DataFrame(columns=['id_produto', 'regiao', 'frete', 'tempo_entrega', 'qtd_vendida', 'avaliacao', 'qtd_fotos', 'tam_desc', 'categoria'])

In [3]:
qtd_pedidos = len(df_orders) 
qtd_avaliacoes = len(df_reviews)
print("qtd_pedidos: %d, qtd_avaliacoes: %d" % (qtd_pedidos, qtd_avaliacoes))


# verificando df_reviews
orderIds =[]
orderIdRepetido = []
for index, row in df_reviews.iterrows():
    oID = row['order_id']
    if oID not in orderIds:
        orderIds.append(oID)
    else:
        orderIdRepetido.append(oID)

print('Total de avaliacoes repetidas: %d, \nTotal de pedidos com avaliacoes: %d' % (len(orderIdRepetido), len(orderIds)))

# Conclusão: todos pedidos tem pelo menos uma avaliacao

qtd_pedidos: 99441, qtd_avaliacoes: 100000
Total de avaliacoes repetidas: 559, 
Total de pedidos com avaliacoes: 99441


In [4]:
orderId_productId = {} # dicionario: chave = order_id, valor = product_id(tupla)

for index, row in df_order_items.iterrows():
    if row['order_id'] not in orderId_productId:
        orderId_productId.update({row['order_id']: (row['product_id'],)})
    else:
        values = orderId_productId.get(row['order_id'])
        orderId_productId.update({row['order_id']:values+(row['product_id'],)})

In [5]:
regiao = {'Sul': ['RS', 'SC', 'PR'], 'Sudeste': ['SP', 'RJ', 'MG', 'ES'], 'Nordeste': ['BA', 'PI', 'MA', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE'], 'Norte': ['TO', 'PA', 'AP', 'RR', 'AM', 'AC', 'RO'], 'Centro-Oeste': ['MT', 'GO', 'MD', 'DF']}

# preenchendo df
# todos os produtos - regiao sul
# todos os produtos - regiao sudestes
# ...
idx = 0
for r in regiao.keys():
    for index, row in df_products.iterrows():
        df.loc[idx, 'id_produto'] = row['product_id']
        df.loc[idx, 'regiao'] = r
        idx+=1

In [6]:
print(len(df_products)*5)
df

164755


,id_produto,regiao,frete,tempo_entrega,qtd_vendida,avaliacao,qtd_fotos,tam_desc,categoria
0,1e9e8ef04dbcff4541ed26657ea517e5,Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3aa071139cb16b67ca9e5dea641aaa2f,Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,96bd76ec8810374ed1b65e291975717f,Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cef67bcfe19066a932b7673e239eb23d,Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9dc1a7de274444849c219cff195d0b71,Sul,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
164750,a0b7d5a992ccda646f2d34e418fff5a0,Centro-Oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164751,bf4538d88321d0fd4412a93c974510e6,Centro-Oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164752,9a7c6041fa9592d9d9ef6cfe62a71f8c,Centro-Oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164753,83808703fc0706a22e264b9d75f04a2e,Centro-Oeste,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
orderID_customerID = {} # dicionario: chave = order_id, valor = customer_id
for index, row in df_orders.iterrows():
    orderID_customerID.update({row['order_id']: row['customer_id']})

orderID_customerID = {} # dicionario: chave = order_id, valor = customer_id
for index, row in df_orders.iterrows():
    orderID_customerID.update({row['order_id']: row['customer_id']})

customerID_regiao = {} # dicionario: chave = customer_id, valor = regiao
for index, row in df_customers.iterrows():
    for r, e in regiao.items(): 
        if row['customer_state'] in e: 
            customerID_regiao.update({row['customer_id']: r})
    
orderID_regiao = {} # dicionario: chave = order_id, valor = regiao
for oID, cID in orderID_customerID.items():
    r = customerID_regiao.get(cID)
    orderID_regiao.update({oID: r})

In [8]:
productId_ordersID = {} # dicionario: chave = id_produto, valor = (id_pedidos)
for index, row in df_products.iterrows():
    prodID = row['product_id'] 
    for pedID, tProd in orderId_productId.items(): 
        if prodID in tProd:
            if prodID not in productId_ordersID:
                productId_ordersID.update({prodID: (pedID,)})
            else:
                values = productId_ordersID.get(prodID)
                productId_ordersID.update({prodID:values+(pedID,)})

In [9]:
orderIDproductID_fretes = {} #dicionario: chave (id_pedido, id_produto), valor = (frete)

for index, row, in df_order_items.iterrows():
    chave = (row['order_id'], row['product_id'])
    if chave not in orderIDproductID_fretes:
        orderIDproductID_fretes.update({chave: (row['freight_value'],)})
    else:
        values = orderIDproductID_fretes.get(chave)
        orderIDproductID_fretes.update({chave:values+(row['freight_value'],)})

orderIDproductID_freteMedia = {}  #dicionario: chave (id_pedido, id_produto), valor = media do frete
for pedIDprodID, fretes in orderIDproductID_fretes.items():
    media = sum(fretes)/len(fretes)
    orderIDproductID_freteMedia.update({pedIDprodID: media})

In [11]:
# frete por regiao
import numpy as np

productID_frete_SUL      = {} #dicionario: chave = id_produto, valor = media do frete
productID_frete_SUDESTE  = {}
productID_frete_NORDESTE = {}
productID_frete_NORTE    = {}
productID_frete_CO       = {}

for index, row in df_products.iterrows():
    prodID = row['product_id']
    pedIDS = productId_ordersID.get(prodID) #tupla com id de pedidos desse produto
    
    # listas com id de pedidos de cada regiao
    pedIDS_SUL, pedIDS_SUDESTE, pedIDS_NORDESTE, pedIDS_NORTE, pedIDS_CO = [], [], [], [], []
    for p in pedIDS:
        if orderID_regiao.get(p) == 'Sul': pedIDS_SUL.append(p)
        if orderID_regiao.get(p) == 'Sudeste': pedIDS_SUDESTE.append(p)
        if orderID_regiao.get(p) == 'Nordeste': pedIDS_NORDESTE.append(p)
        if orderID_regiao.get(p) == 'Norte': pedIDS_NORTE.append(p)
        if orderID_regiao.get(p) == 'Centro-Oeste': pedIDS_CO.append(p)

    # listas com os valores de frete para produtos de pedidos de cada regiao
    f_SUL, f_SUDESTE, f_NORDESTE, f_NORTE, f_CO = [], [], [], [], []
    for p in pedIDS_SUL: f_SUL.append(orderIDproductID_freteMedia.get((p, prodID)))
    for p in pedIDS_SUDESTE: f_SUDESTE.append(orderIDproductID_freteMedia.get((p, prodID)))
    for p in pedIDS_NORDESTE: f_NORDESTE.append(orderIDproductID_freteMedia.get((p, prodID)))
    for p in pedIDS_NORTE: f_NORTE.append(orderIDproductID_freteMedia.get((p, prodID)))
    for p in pedIDS_CO: f_CO.append(orderIDproductID_freteMedia.get((p, prodID)))
        
    # atualizando o dicionario com a média do valor de frete para tal produto em tal regiao
    productID_frete_SUL.update({prodID: np.mean(f_SUL)})
    productID_frete_SUDESTE.update({prodID: np.mean(f_SUDESTE)})
    productID_frete_NORDESTE.update({prodID: np.mean(f_NORDESTE)})
    productID_frete_NORTE.update({prodID: np.mean(f_NORTE)})
    productID_frete_CO.update({prodID: np.mean(f_CO)})


C:\Users\leuze\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\leuze\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
# preenche a coluna frete da tabela de acordo com a media de frete de tal produto em tal regiao
idx = 0
for r in regiao.keys():
        for index, row in df_products.iterrows():
            prodID = row['product_id']
            if r == 'Sul':          df.loc[idx, 'frete'] = productID_frete_SUL.get(prodID)
            if r == 'Sudeste':      df.loc[idx, 'frete'] = productID_frete_SUDESTE.get(prodID)
            if r == 'Nordeste':     df.loc[idx, 'frete'] = productID_frete_NORDESTE.get(prodID)
            if r == 'Norte':        df.loc[idx, 'frete'] = productID_frete_NORTE.get(prodID)
            if r == 'Centro-Oeste': df.loc[idx, 'frete'] = productID_frete_CO.get(prodID)
            idx+=1

In [13]:
import datetime

# funcao que calcula a diferenca entre 2 datas:
# data em que cliente comprou e data estimada de entrega mostrada p cliente qnd ele comprou 
# resultado da diferenca eh retornado em dias
def tempoEntregaEstimado(date1, date2):
    date1 = date1.split(' ')[0]
    date1Vet = date1.split('-')
    date2 = date2.split(' ')[0]
    date2Vet = date2.split('-')

    # datetime(year, month, day, hour, minute, second)
    d1 = datetime.datetime(int(date1Vet[0]), int(date1Vet[1]), int(date1Vet[2]))
    d2 = datetime.datetime(int(date2Vet[0]), int(date2Vet[1]), int(date2Vet[2]))
    diferenca = abs(d1-d2) 
    
    return diferenca.days

In [14]:
orderIDcustomerID_tempo = {} #dicionario: chave (id_pedido, id_cliente), valor = (tempo)
for index, row, in df_orders.iterrows():
    chave = (row['order_id'], row['customer_id'])
    tempoEstimado = tempoEntregaEstimado(row['order_estimated_delivery_date'], row['order_estimated_delivery_date'])
    if chave not in orderIDcustomerID_tempo:
        orderIDcustomerID_tempo.update({chave: (tempoEstimado,)})
    else:
        values = orderIDcustomerID_tempo.get(chave)
        orderIDcustomerID_tempo.update({chave:values+(tempoEstimado,)})

orderIDcustomerID_tempoMedia = {}  #dicionario: chave (id_pedido, id_cliente), valor = media do tempo
for pedIDcliID, tempos in orderIDcustomerID_tempo.items():
    media = sum(tempos)/len(tempos)
    orderIDcustomerID_tempoMedia.update({pedIDcliID: media})

In [15]:
# estimativa de tempo de entrega por regiao
import numpy as np

productID_tempo_SUL      = {} #dicionario: chave = id_produto, valor = media do tempo
productID_tempo_SUDESTE  = {}
productID_tempo_NORDESTE = {}
productID_tempo_NORTE    = {}
productID_tempo_CO       = {}
for index, row in df_products.iterrows():
    prodID = row['product_id']
    pedIDS = productId_ordersID.get(prodID) #tupla com id de pedidos desse produto
    
    # listas com ids de pedidos de cada regiao para tal produto
    pedIDS_SUL, pedIDS_SUDESTE, pedIDS_NORDESTE, pedIDS_NORTE, pedIDS_CO = [], [], [], [], []
    for p in pedIDS:
        if orderID_regiao.get(p) == 'Sul': pedIDS_SUL.append(p)
        if orderID_regiao.get(p) == 'Sudeste': pedIDS_SUDESTE.append(p)
        if orderID_regiao.get(p) == 'Nordeste': pedIDS_NORDESTE.append(p)
        if orderID_regiao.get(p) == 'Norte': pedIDS_NORTE.append(p)
        if orderID_regiao.get(p) == 'Centro-Oeste': pedIDS_CO.append(p)

    # listas com as estimativas de entrega dos produtos em cada regiao
    tempo_SUL, tempo_SUDESTE, tempo_NORDESTE, tempo_NORTE, tempo_CO = [], [], [], [], []
    for p in pedIDS_SUL: 
        cliID = orderID_customerID.get(p)
        tempo_SUL.append(orderIDcustomerID_tempoMedia.get((p, cliID)))
    for p in pedIDS_SUDESTE: 
        cliID = orderID_customerID.get(p)
        tempo_SUDESTE.append(orderIDcustomerID_tempoMedia.get((p, cliID)))
    for p in pedIDS_NORDESTE: 
        cliID = orderID_customerID.get(p)
        tempo_NORDESTE.append(orderIDcustomerID_tempoMedia.get((p, cliID)))
    for p in pedIDS_NORTE: 
        cliID = orderID_customerID.get(p)
        tempo_NORTE.append(orderIDcustomerID_tempoMedia.get((p, cliID)))
    for p in pedIDS_CO: 
        cliID = orderID_customerID.get(p)
        tempo_CO.append(orderIDcustomerID_tempoMedia.get((p, cliID)))
        
    # atualizando os dicionarios com a media de tempos de entrega para cada regiao
    productID_tempo_SUL.update({prodID: np.mean(tempo_SUL)})
    productID_tempo_SUDESTE.update({prodID: np.mean(tempo_SUDESTE)})
    productID_tempo_NORDESTE.update({prodID: np.mean(tempo_NORDESTE)})
    productID_tempo_NORTE.update({prodID: np.mean(tempo_NORTE)})
    productID_tempo_CO.update({prodID: np.mean(tempo_CO)})



In [16]:
# preenche a coluna tempo_entrega da tabela de acordo com a media de tempo de tal produto em tal regiao
idx = 0
for r in regiao.keys():
        for index, row in df_products.iterrows():
            prodID = row['product_id']
            if r == 'Sul':          df.loc[idx, 'tempo_entrega'] = productID_tempo_SUL.get(prodID)
            if r == 'Sudeste':      df.loc[idx, 'tempo_entrega'] = productID_tempo_SUDESTE.get(prodID)
            if r == 'Nordeste':     df.loc[idx, 'tempo_entrega'] = productID_tempo_NORDESTE.get(prodID)
            if r == 'Norte':        df.loc[idx, 'tempo_entrega'] = productID_tempo_NORTE.get(prodID)
            if r == 'Centro-Oeste': df.loc[idx, 'tempo_entrega'] = productID_tempo_CO.get(prodID)
            idx+=1

In [18]:
# colunas de df que independem da regiao, serão replicada me blocos na tabela
from collections import Counter
# qtd_vendida
productID_vendidos = Counter(df_order_items['product_id']) # dicionario: chave = id_produto, valor = qtd de vendidos

# avaliacao
productId_reviewScore = {} # dicionario: chave = product_id, valor = media da avaliacao
for order in orderIds:
    scores = df_reviews.loc[df_reviews['order_id'] == order]['review_score']
    soma = 0
    for valor in scores: soma += valor
    media = soma / len(scores) # media das avaliacoes do pedido
    produtos = orderId_productId.get(order)
    if produtos != None: #tem um pedido que nao tem produtos associados
        for p in produtos:
            if p not in productId_reviewScore:
                productId_reviewScore.update({p: media})
            else:
                anterior = productId_reviewScore.get(p)
                productId_reviewScore.update({p: (anterior+media)/2})

# qtd_fotos
productID_fotos = {} # dicionario: chave = id_produto, valor = qtd de fotos
# tam_desc
productID_descricao = {} # dicionario: chave = id_produto, valor = tamanho da descricao
# categoria
productID_categoria = {} # dicionario: chave = id_produto, valor = categoria
for index, row in df_products.iterrows():
    prodID = row['product_id']
    productID_fotos.update({prodID: row['product_photos_qty']})
    productID_descricao.update({prodID: row['product_description_lenght']})
    productID_categoria.update({prodID: row['product_category_name']})

In [19]:
# preenche as colunas qtd_vendida, avaliacao, qtd_fotos, tam_desc, categoria tabela de acordo com os dados do produto 
idx = 0
for r in regiao.keys():
        for index, row in df_products.iterrows():
            prodID = row['product_id']
            df.loc[idx, 'qtd_vendida'] = productID_vendidos.get(prodID)
            df.loc[idx, 'avaliacao']   = productId_reviewScore.get(prodID)
            df.loc[idx, 'qtd_fotos']   = productID_fotos.get(prodID)
            df.loc[idx, 'tam_desc']    = productID_descricao.get(prodID)
            df.loc[idx, 'categoria']   = productID_categoria.get(prodID)
            idx+=1

In [20]:
df

,id_produto,regiao,frete,tempo_entrega,qtd_vendida,avaliacao,qtd_fotos,tam_desc,categoria
0,1e9e8ef04dbcff4541ed26657ea517e5,Sul,NaN,NaN,1,5,1,287,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,Sul,NaN,NaN,1,5,1,276,artes
2,96bd76ec8810374ed1b65e291975717f,Sul,NaN,NaN,1,5,1,250,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,Sul,NaN,NaN,1,1,1,261,bebes
4,9dc1a7de274444849c219cff195d0b71,Sul,NaN,NaN,1,5,4,402,utilidades_domesticas
...,...,...,...,...,...,...,...,...,...
164750,a0b7d5a992ccda646f2d34e418fff5a0,Centro-Oeste,NaN,NaN,33,3.22363,2,67,moveis_decoracao
164751,bf4538d88321d0fd4412a93c974510e6,Centro-Oeste,NaN,NaN,2,4.5,1,971,construcao_ferramentas_iluminacao
164752,9a7c6041fa9592d9d9ef6cfe62a71f8c,Centro-Oeste,NaN,NaN,5,4.4375,1,799,cama_mesa_banho
164753,83808703fc0706a22e264b9d75f04a2e,Centro-Oeste,NaN,NaN,7,4.96875,2,156,informatica_acessorios


In [24]:
df.to_csv("dados_v1.csv", index=None)

## Observações

+ Todos pedidos tem pelo menos uma avaliacao

+ Exemplo de pedido que nao tem produto associado - hipotese: Cancelaram a compra

In [21]:
print(orderId_productId.get('534a0fbe91a806a1dc6bf09b497e3d6f'))
df_reviews.loc[df_reviews['order_id'] == "534a0fbe91a806a1dc6bf09b497e3d6f"] #existe avaliacao associada a pedido

None


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
307,6280ca6a9e8c3a56b668ec8c5396caaf,534a0fbe91a806a1dc6bf09b497e3d6f,1,NaN,Passou dos dias previstos.,2018-02-09 00:00:00,2018-02-14 04:13:49


In [22]:
df_order_items.loc[df_order_items['order_id'] == "534a0fbe91a806a1dc6bf09b497e3d6f"] #nao existe produto associado ao pedido

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value


## Visualizações

In [23]:
# grafico: pedidos x qtd de avaliacoes do pedido
from collections import Counter
import plotly.express as px

qtdReviews_orders = Counter(orderIdRepetido+orderIds) #dicionario: chave = order_id, valor = qtd de avaliacoes
qtd = Counter(qtdReviews_orders.values()) #dicionario: chave = qtd de avaliacoes, valor = qtd de pedidos


df_pedidos_avaliacoes = pd.DataFrame(columns=["Avaliações", 'Pedidos'])
df_pedidos_avaliacoes['Pedidos'] = qtd.values()
df_pedidos_avaliacoes['Avaliações'] = qtd.keys()
fig = px.scatter(df_pedidos_avaliacoes, y="Avaliações", x="Pedidos", title="Quantidade de avaliações X Pedidos")
fig.show()
df_pedidos_avaliacoes

,Avaliações,Pedidos
0,2,551
1,3,4
2,1,98886
